`lblnew.f` takes as one of its inputs a file that contains the atmosphere profile for which radiative transfer is to be calculated. For example, the profile can be like that found in mid-latitude summer, or a sub-arctic winter.  

Here, we try to make an atmosphere profile that is not necessarily like a realistic profile found in some part of the Earth, but contains layers whose pressure and temperature cover ranges specified by us.  This allows as to construct tables of absorption coefficient values from which we could later look up.

In [77]:
import os
import sys
import collections
import itertools

import re

import numpy as np
import pandas as pd


In [80]:
def pattern_data(name):                                                                
    '''                                                                                
    Returns regular expression for a Fortran                                           
    'data' variable assignment                                                         
    '''                                                                                
    return '''                                                                         
    (data [^/]+ {name} [^/]+ / ([^/]+) /)                              
    '''.format(name=name)


def numbers(s=''):                                                                     
    '''                                                                                
    Extract numebers from a piece of Fortran code.  This covers                        
    numbers written in various forms, such as 8, .8, 8., -8E-09,                       
    8e-09.                                                                             
                                                                                       
    Parameters                                                                         
    ----------                                                                         
    s : {string}                                                                       
        Fortran code                                                                   
    '''                                                                                
    pattern = '''                                                                      
    (                                                                                  
    -?                                                                                 
    \d+                                                                                
    (?: \. \d*)?                                                                       
    (?: (?: E|e) (?: -|\+) \d+)?                                                       
    )                                                                                  
    (?: _r8)?                                                                          
    '''                                                                                
    regex = re.compile(pattern, re.VERBOSE)                                            
    return regex.findall(s)     

In [82]:
def mls75_r8():
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation',
                         'crd/atmosphere_profiles',
                         'mls75_r8.pro')

    with open(fpath, mode='r', encoding='utf-8') as f:
        c = f.read()        
    return c


def saw75_r8():
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation',
                         'crd/atmosphere_profiles',
                         'saw75_r8.pro')

    with open(fpath, mode='r', encoding='utf-8') as f:
        c = f.read()
    return c


def trp75_r8():
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation',
                         'crd/atmosphere_profiles',
                         'trp75_r8.pro')

    with open(fpath, mode='r', encoding='utf-8') as f:
        c = f.read()
    return c

In [93]:
def get_var(s, name):
    regex = re.compile(pattern_data(name), re.VERBOSE)
    regex_find = regex.findall(c)
    data_statement, data_content = regex_find[0]
    
    var = np.array(numbers(data_content), dtype='float')
    return var

In [118]:
def tmp():

    # Find the minimum and maximum pressures from
    # all of mls, saw and trp profiles
    plevels = np.array(
        [
            get_var(mls75_r8(), 'plevel'),
            get_var(saw75_r8(), 'plevel'),
            get_var(trp75_r8(), 'plevel')
            ])

    print('minimum pressure =', plevels.min(), 'mb')
    print('maximum pressure =', plevels.max(), 'mb')


    # Find the minimum and maximum temperatures from
    # all of mls, saw and trp profiles
    tlayers = np.array(
        [
            get_var(mls75_r8(), 'tlayer'),
            get_var(saw75_r8(), 'tlayer'),
            get_var(trp75_r8(), 'tlayer')
        ]
    )

    print('minimum temperature =', tlayers.min(), 'K')
    print('maximum temperature =', tlayers.max(), 'K')


def table_pres():
    return np.array([10., 20, 30])

 
def table_temp():
    return np.array([260, 270])


def atmpro_vars(unique_pres, unique_temp):
    atmpro_varnames = ['player', 'tlayer', 'wlayer', 'olayer']
    
    if len(unique_pres) != len(set(unique_pres)):
        raise ValueError('Input pressure values must be unique.')
        
    if len(unique_temp) != len(set(unique_temp)):
        raise ValueError('Input temperature values must be unique.')
        
    pres, temp = zip(*itertools.product(unique_pres, unique_temp))
    
    pres = np.array(pres)
    temp = np.array(temp)
    
    nlayer = len(pres)
    
    wlayer = np.zeros(pres.shape)
    olayer = np.zeros(pres.shape)
    
    
    return dict(zip(atmpro_varnames, [pres, temp, wlayer, olayer]))



def atmpro_player(dict_atmpro, saveas=None):
    '''
    This writes out a .pro file with `player`.
    '''
    varnames = ['player', 'tlayer', 'wlayer', 'olayer']
    
    s_data = '\n'.join(
        [
            '      data ({name}(i), i = 1, nlayer) /',
            '{values} / \n\n'
        ])
    
    c = ''
    for name in varnames:
        vs = dict_atmpro[name]
        vs = [str(v) + '_r8' for v in vs]
        s_vs = ',\n     &     '.join(vs)
        s_vs = '     &     ' + s_vs
        s = s_data.format(name=name, values=s_vs)
        c += s
    
    c = '!\n!\n!\n' + c
    
    if saveas != None:
        with open(saveas, mode='w', encoding='utf-8') as f:
            f.write(c)
            
    return c
        
        
        
    
    
    
    
    
dict_atmpro = atmpro_vars(table_pres(), table_temp())


saveas = os.path.join('/chia_cluster/home/jackyu/radiation',
                      'crd/atmosphere_profiles',
                      'table_clirad_example.pro')

atmpro_player(dict_atmpro, saveas=saveas)

'!\n!\n!\n      data (player(i), i = 1, nlayer) /\n     &     10.0_r8,\n     &     10.0_r8,\n     &     20.0_r8,\n     &     20.0_r8,\n     &     30.0_r8,\n     &     30.0_r8 / \n\n      data (tlayer(i), i = 1, nlayer) /\n     &     260_r8,\n     &     270_r8,\n     &     260_r8,\n     &     270_r8,\n     &     260_r8,\n     &     270_r8 / \n\n      data (wlayer(i), i = 1, nlayer) /\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8 / \n\n      data (olayer(i), i = 1, nlayer) /\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8,\n     &     0.0_r8 / \n\n'

In [71]:
# Load plevel from mls75_r8.pro.
# 

c = mls75_r8()

# player
regex = re.compile(pattern_data('plevel'), re.VERBOSE)
regex_find = regex.findall(c)
data_statement, data_content = regex_find[0]

plevel = np.array(numbers(data_content), dtype='float')

player = .5 * (plevel[:-1] + plevel[1:])

player = [str(v) + '_r8' for v in player]

#print(data_content)

new_data_content = '\n     &    ' + ',\n     &    '.join(player)

new_data_statement = data_statement.replace('plevel', 'player')
new_data_statement = new_data_statement.replace('nlayer+1', 'nlayer')
new_data_statement = new_data_statement.replace(data_content,
                                                new_data_content)

new_c = c.replace(data_statement, new_data_statement)

with open(os.path.join('/chia_cluster/home/jackyu/radiation',
                       'crd/atmosphere_profiles',
                       'mls75_player_r8.pro'),
          mode='w', encoding='utf-8') as f:
    f.write(new_c)
    


In [148]:
def rrtmg_ktable_pts():
    fpath = os.path.join('/chia_cluster/home/jackyu',
                         'radiation/offline_radiation_notebooks',
                         'trace_rrtmg/rrtmg_pts_for_ktable.csv')
    return pd.read_csv(fpath, index_col=[0])



def get_dict_atmpro():
    df = rrtmg_ktable_pts().set_index('pref')

    pts = []
    for pref, temps in df.iterrows():
        pts.extend(list(itertools.product([pref], temps)))
    
    player, tlayer = itertools.zip_longest(*pts)
    
    player = np.array(player)
    tlayer = np.array(tlayer)
    wlayer = np.zeros(player.shape)
    olayer = np.zeros(player.shape)
    
    return {'player': player, 
            'tlayer': tlayer, 
            'wlayer': wlayer, 
            'olayer': olayer}

s = atmpro_player(get_dict_atmpro(), saveas='rrtmg_ktable_pts_atmpro.pro')